In [1]:
%run prepareStats.py

Last GP in the database: 2022 Hungarian Grand Prix


In [2]:
fathersAndSons = {
"Andrettis": ["Mario Andretti", "Michael Andretti"],
"Brabhams": ["Jack Brabham", "David Brabham"],
"Fittipaldis": ["Wilson Fittipaldi", "Christian Fittipaldi"],
"Hills": ["Graham Hill", "Damon Hill"], 
"Magnussens": ["Jan Magnussen", "Kevin Magnussen"],
"Nakajimas": ["Satoru Nakajima", "Kazuki Nakajima"],
"Palmers": ["Jonathan Palmer", "Jolyon Palmer"],
"Parnells": ["Reg Parnell", "Tim Parnell"],
"Piquets": ["Nelson Piquet", "Nelson Piquet Jr."], 
"Rosbergs": ["Keke Rosberg", "Nico Rosberg"],
"Schumachers": ["Michael Schumacher", "Mick Schumacher"],
"Verstappens": ["Jos Verstappen", "Max Verstappen"], 
"Villeneuves": ["Gilles Villeneuve", "Jacques Villeneuve"],
"Winkelhocks": ["Manfred Winkelhock", "Markus Winkelhock"]
}

In [3]:
len(fathersAndSons)

14

There have been 14 F1 drivers that are sons of another F1 driver. By participating in 2 races in 2020, [Pietro Fittipaldi](https://en.wikipedia.org/wiki/Pietro_Fittipaldi) became the first (and still the only) competing grandson of another driver.

In [4]:
sons = []
for key, value in fathersAndSons.items():
    sons.append(value[1])

In [5]:
results.loc[results.name.isin(sons),"son"]=True

In [6]:
results.loc[results.position.eq(1),"twentyFivePointSystem"]=25
results.loc[results.position.eq(2),"twentyFivePointSystem"]=18
results.loc[results.position.eq(3),"twentyFivePointSystem"]=15
results.loc[results.position.eq(4),"twentyFivePointSystem"]=12
results.loc[results.position.eq(5),"twentyFivePointSystem"]=10
results.loc[results.position.eq(6),"twentyFivePointSystem"]=8
results.loc[results.position.eq(7),"twentyFivePointSystem"]=6
results.loc[results.position.eq(8),"twentyFivePointSystem"]=4
results.loc[results.position.eq(9),"twentyFivePointSystem"]=2
results.loc[results.position.eq(10),"twentyFivePointSystem"]=1
results.loc[~results.position.isin([1,2,3,4,5,6,7,8,9,10]),"twentyFivePointSystem"]=0

In [7]:
resultsSons = results[results.name.isin(sons)]

## Historical stats

In [8]:
woSons = results.sort_values(by="date").groupby(["year", "round", "location"])["son"].count()
woSons[woSons <1].tail(1)

year  round  location
2004  15     Monza       0
Name: son, dtype: int64

The last GP without a single son of another F1 driver entering was the 2004 Italian Grand Prix.

In [9]:
woSons = results.sort_values(by="date").groupby(["year"])["son"].count()
woSons[woSons <1].tail(1)

year
1991    0
Name: son, dtype: int64

In [10]:
results[results.name.isin(sons)].reset_index().groupby("year")["name"].nunique(list)

year
1959    1
1961    1
1963    1
1990    1
1992    2
1993    3
1994    3
1995    1
1996    2
1997    2
1998    2
1999    2
2000    1
2001    1
2002    1
2003    1
2004    1
2005    1
2006    2
2007    3
2008    3
2009    3
2010    1
2011    1
2012    1
2013    1
2014    2
2015    3
2016    4
2017    3
2018    2
2019    2
2020    2
2021    2
2022    3
Name: name, dtype: int64

The 2016 season saw the most sons of another F1 driver participating in the WDC.

In [11]:
resultsSons[resultsSons["year"] == 2016].groupby("name")["raceId"].nunique()

name
Jolyon Palmer      21
Kevin Magnussen    21
Max Verstappen     21
Nico Rosberg       21
Name: raceId, dtype: int64

## Sons, ranked

In [12]:
resultsSons.groupby("name")["points"].sum().sort_values(ascending=False)

name
Max Verstappen          1792.5
Nico Rosberg            1594.5
Damon Hill               360.0
Jacques Villeneuve       235.0
Kevin Magnussen          177.0
Nelson Piquet Jr.         19.0
Christian Fittipaldi      12.0
Mick Schumacher           12.0
Jolyon Palmer              9.0
Kazuki Nakajima            9.0
Michael Andretti           7.0
David Brabham              0.0
Markus Winkelhock          0.0
Tim Parnell                0.0
Name: points, dtype: float64

In [13]:
resultsSons.groupby("name")["twentyFivePointSystem"].sum().sort_values(ascending=False)

name
Max Verstappen          1791.0
Nico Rosberg            1739.0
Damon Hill              1091.0
Jacques Villeneuve       853.0
Kevin Magnussen          177.0
Christian Fittipaldi     100.0
Nelson Piquet Jr.         50.0
Michael Andretti          41.0
Kazuki Nakajima           34.0
Mick Schumacher           12.0
Jolyon Palmer              9.0
David Brabham              1.0
Tim Parnell                1.0
Markus Winkelhock          0.0
Name: twentyFivePointSystem, dtype: float64

In [14]:
resultsSons[resultsSons["position"] == 1].groupby("name")["raceId"].nunique().sort_values(ascending=False)

name
Max Verstappen        28
Nico Rosberg          23
Damon Hill            22
Jacques Villeneuve    11
Name: raceId, dtype: int64

In [15]:
resultsSons.groupby("name")["raceId"].count().sort_values(ascending=False)

name
Nico Rosberg            206
Jacques Villeneuve      165
Max Verstappen          154
Kevin Magnussen         133
Damon Hill              122
Christian Fittipaldi     43
Jolyon Palmer            37
Kazuki Nakajima          36
Mick Schumacher          35
David Brabham            30
Nelson Piquet Jr.        28
Michael Andretti         13
Tim Parnell               4
Markus Winkelhock         1
Name: raceId, dtype: int64

In [16]:
resultsSons.groupby("name")["position"].min().sort_values()

name
Damon Hill               1.0
Jacques Villeneuve       1.0
Max Verstappen           1.0
Nico Rosberg             1.0
Kevin Magnussen          2.0
Nelson Piquet Jr.        2.0
Michael Andretti         3.0
Christian Fittipaldi     4.0
Jolyon Palmer            6.0
Kazuki Nakajima          6.0
Mick Schumacher          6.0
David Brabham           10.0
Tim Parnell             10.0
Markus Winkelhock        NaN
Name: position, dtype: float64

In [17]:
resultsSons[resultsSons["grid"] == 1].groupby("name")["raceId"].count().sort_values(ascending=False)

name
Nico Rosberg          30
Damon Hill            20
Max Verstappen        16
Jacques Villeneuve    13
Name: raceId, dtype: int64

## Gaps between father's last GP and son's first

In [18]:
def timedelta():
    timedeltas = {}
    for key, value in fathersAndSons.items():
        delta = results[results["name"] == value[1]]["date"].min() - results[results["name"] == value[0]]["date"].max()
        delta = delta
        timedeltas[key] = delta
    timedeltas = pd.DataFrame.from_dict(timedeltas, orient="index", columns = ["gap"])    
    return(timedeltas)
        
timedelta().sort_values(by="gap")

,gap
Parnells,1827 days
Schumachers,3045 days
Andrettis,3823 days
Verstappens,4172 days
Villeneuves,5054 days
Magnussens,5761 days
Nakajimas,5831 days
Piquets,5978 days
Fittipaldis,5992 days
Hills,6202 days


## Drivers both father and son have raced against

In [19]:
results["adversaries"] = results["raceId"].map(results.groupby("raceId")["name"].agg(list))

In [20]:
def intersection(lst1, lst2):
    lst3 = [value for value in lst1 if value in lst2]
    return lst3

def adversaries():
    adversaries = {}
    for key, value in fathersAndSons.items():
        father = results[results["name"] == value[0]]
        father = father["adversaries"].tolist()
        father = [item for sublist in father for item in sublist]
        father = list(set(father))
        son = results[results["name"] == value[1]]
        son = son["adversaries"].tolist()
        son = [item for sublist in son for item in sublist]
        son = list(set(son))        
        inter = intersection(father, son)
        pair = [len(inter), ", ".join(inter)]
        adversaries[key] = pair
    pd.set_option('display.max_colwidth', None)
    adversaries = pd.DataFrame.from_dict(adversaries, orient="index", columns=["count", "names"])    
    
    return(adversaries)

adversaries().sort_values(by="count", ascending = False)

,count,names
Schumachers,8,"Lewis Hamilton, Sebastian Vettel, Fernando Alonso, Robert Kubica, Nico Hülkenberg, Daniel Ricciardo, Kimi Räikkönen, Sergio Pérez"
Parnells,6,"Harry Schell, Maurice Trintignant, Stirling Moss, André Pilette, Ron Flockhart, Roy Salvadori"
Andrettis,5,"Michele Alboreto, Alain Prost, Riccardo Patrese, Andrea de Cesaris, Derek Warwick"
Verstappens,3,"Fernando Alonso, Jenson Button, Kimi Räikkönen"
Brabhams,0,
Fittipaldis,0,
Hills,0,
Magnussens,0,
Nakajimas,0,
Palmers,0,


## Circuits both father and son have raced (and won) at

In [21]:
results["tracks"] = results["name"].map(results.groupby("name")["location"].agg(list))

In [22]:
def intersection(lst1, lst2):
    lst3 = [value for value in lst1 if value in lst2]
    return lst3

def tracks():
    tracks = {}
    for key, value in fathersAndSons.items():
        father = results[results["name"] == value[0]]
        father = father["tracks"].tolist()
        father = [item for sublist in father for item in sublist]
        father = list(set(father))
        son = results[results["name"] == value[1]]
        son = son["tracks"].tolist()
        son = [item for sublist in son for item in sublist]
        son = list(set(son))        
        inter = intersection(father, son)
        pair = [len(inter), ", ".join(inter)]
        tracks[key] = pair
    pd.set_option('display.max_colwidth', None)
    tracks = pd.DataFrame.from_dict(tracks, orient="index", columns=["count", "tracks"])    
    
    return(tracks)

tracks().sort_values(by="count", ascending = False)

,count,tracks
Schumachers,16,"Monte-Carlo, Spa, São Paulo, Mexico City, Spielberg, Sakhir, Austin, Istanbul, Montmeló, Melbourne, Abu Dhabi, Montreal, Monza, Silverstone, Budapest, Imola"
Verstappens,15,"Monte-Carlo, Spa, Nürburg, São Paulo, Spielberg, Montmeló, Hockenheim, Suzuka, Melbourne, Montreal, Monza, Silverstone, Kuala Lumpur, Imola, Budapest"
Magnussens,14,"Monte-Carlo, Montmeló, Silverstone, Spa, Hockenheim, Suzuka, Spielberg, Melbourne, São Paulo, Imola, Nürburg, Montreal, Monza, Budapest"
Rosbergs,12,"Monte-Carlo, Spa, Nürburg, São Paulo, Mexico City, Spielberg, Hockenheim, Montreal, Monza, Silverstone, Imola, Budapest"
Nakajimas,11,"Monte-Carlo, Spa, São Paulo, Montmeló, Hockenheim, Suzuka, Montreal, Monza, Silverstone, Magny Cours, Budapest"
Piquets,11,"Monte-Carlo, Spa, Nürburg, São Paulo, Montmeló, Hockenheim, Montreal, Monza, Silverstone, Magny Cours, Budapest"
Hills,10,"Monte-Carlo, Spa, Nürburg, São Paulo, Spielberg, Midrand, Hockenheim, Monza, Silverstone, Buenos Aires"
Palmers,10,"Monte-Carlo, Spa, Mexico City, Spielberg, Hockenheim, Suzuka, Montreal, Monza, Silverstone, Budapest"
Villeneuves,9,"Monte-Carlo, São Paulo, Spielberg, Hockenheim, Montreal, Monza, Silverstone, Buenos Aires, Imola"
Andrettis,8,"Monte-Carlo, São Paulo, Midrand, Hockenheim, Montreal, Monza, Silverstone, Imola"


In [23]:
wins["tracks"] = wins["name"].map(wins.groupby("name")["location"].agg(list))

C:\Users\micha\AppData\Local\Temp\ipykernel_6504\1730858596.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wins["tracks"] = wins["name"].map(wins.groupby("name")["location"].agg(list))


In [24]:
def intersection(lst1, lst2):
    lst3 = [value for value in lst1 if value in lst2]
    return lst3

def tracks():
    tracks = {}
    for key, value in fathersAndSons.items():
        father = wins[results["name"] == value[0]]
        father = father["tracks"].dropna().tolist()
        father = [item for sublist in father for item in sublist]
        father = list(set(father))
        son = wins[results["name"] == value[1]]
        son = son["tracks"].dropna().tolist()
        son = [item for sublist in son for item in sublist]
        son = list(set(son))        
        inter = intersection(father, son)
        pair = [len(inter), ", ".join(inter)]
        tracks[key] = pair
    pd.set_option('display.max_colwidth', None)
    tracks = pd.DataFrame.from_dict(tracks, orient="index", columns=["count", "tracks"])    
    
    return(tracks)

tracks().sort_values(by="count", ascending = False)

C:\Users\micha\AppData\Local\Temp\ipykernel_6504\2317356391.py:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  father = wins[results["name"] == value[0]]
C:\Users\micha\AppData\Local\Temp\ipykernel_6504\2317356391.py:12: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  son = wins[results["name"] == value[1]]


,count,tracks
Hills,1,Monza
Rosbergs,1,Monte-Carlo
Andrettis,0,
Brabhams,0,
Fittipaldis,0,
Magnussens,0,
Nakajimas,0,
Palmers,0,
Parnells,0,
Piquets,0,


## How old was the son on the day of his father's last race?

In [25]:
def timedeltaAge():
    timedeltasAge = {}
    for key, value in fathersAndSons.items():
        deltaAge = results[results["name"] == value[0]]["date"].max() - results[results["name"] == value[1]]["dob"].min()
        deltaAge = deltaAge
        timedeltasAge[key] = deltaAge
    timedeltasAge = pd.DataFrame.from_dict(timedeltasAge, orient="index", columns = ["age"])    
    return(timedeltasAge)
        
timedeltaAge().sort_values(by="age")

,age
Palmers,-441 days
Rosbergs,486 days
Fittipaldis,1721 days
Brabhams,1876 days
Winkelhocks,1878 days
Magnussens,2071 days
Verstappens,2203 days
Piquets,2292 days
Nakajimas,2487 days
Villeneuves,4048 days
